In [1]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
# import data_helpers
from tensorflow.contrib import learn
import csv
from pathlib import Path 
import gensim
import pickle
import pandas as pd
from sklearn import preprocessing
import re

In [2]:
data_dir_path = os.path.join(Path(os.getcwd()).parent.parent, 'deeptech-ai-data')

In [3]:
# load model
model_path = os.path.join(data_dir_path, 'german.model')
trained_model = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)
vocab = list(trained_model.vocab)

In [4]:
def get_df_companies_labeling(file):
    with open(file, 'rb') as pickle_file:
        companies_labeling = pickle.load(pickle_file)
    
    df = pd.DataFrame(companies_labeling)
    
    categories = list(set(df['section_name']))
    
    le = preprocessing.LabelEncoder()
    le.fit(df['section'])
    df['y'] = le.transform(df['section'])
    
    return df

In [5]:
companies_labeling_with_cat_num = os.path.join(data_dir_path, 'companies_enchanced_labeling_cat.pickle')
df = get_df_companies_labeling(companies_labeling_with_cat_num)

In [6]:
# df

,url,section,sub_1,sub_2,section_name,sub_1_name,sub_2_name,y
0,www.hoberg-driesch.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
1,www.treureal.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
2,www.immo-ostalb.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
3,www.trauco.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
4,www.kskgrossgerau.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
5,www.bank1saar-immobilien.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
6,www.prehn-bauprojekt.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
7,www.casamundo.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
8,www.preusse-elektro.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11
9,www.coesfeld.de,L,68,31,GRUNDSTÜCKS- UND WOHNUNGSWESEN,Grundstücks- und Wohnungswesen,"Vermittlung von Grundstücken, Gebäuden und Woh...",11


In [7]:
def metadata_df_from_file(file):
    with open(file, 'rb') as pickle_file:
        metadata_df = pickle.load(pickle_file)
        
    return metadata_df

In [8]:
metadata_df_file = os.path.join(data_dir_path, 'metadata_df.pickle')
metadata_df = metadata_df_from_file(metadata_df_file)

In [9]:
# metadata_df

,url,section,code,1,abstract,description,keywords
0,www.hoberg-driesch.de,L,68.31,(Qualitätsstahlrohre von Hoberg & Driesch. Von...,Qualitätsstahlrohre von Hoberg & Driesch. Von ...,Hoberg & Driesch GmbH & Co. KG ist eines der f...,"[Röhrengrosshandel, Starkwandrohre, Normalwand..."
1,www.hgdf.de,K,64.20,"(, , , False)",,,[]
2,www.gasag.de,D,35.30,"(, Bei der GASAG finden Sie die richtigen Stro...",,Bei der GASAG finden Sie die richtigen Strom- ...,"[Strom, Erdgas, Wärme, Privatkunden, Stromvert..."
3,www.ergodirekt.de,K,65.11,"(, ERGO Direkt ist meistgewählter Direktversic...",,ERGO Direkt ist meistgewählter Direktversicher...,[]
4,www.wahler.de,M,70.10,"(, , , False)",,,[]
5,www.msa-auer.de,S,95.11,"(, MSA is a global manufacturer of safety prod...",,MSA is a global manufacturer of safety product...,"[MSA, MSA safety, MSAsafety, safety products, ..."
6,www.nuernberger.de,M,70.10,"(, Die NÜRNBERGER Versicherung - Solide. Finan...",,Die NÜRNBERGER Versicherung - Solide. Finanzst...,"[NÜRNBERGER, Versicherung]"
7,www.ds-cc.de,M,73.11,"(, Wir sind eines der führenden Outsourcing-Un...",,Wir sind eines der führenden Outsourcing-Unter...,"[Customer Care Services, Kundenservices, Kunde..."
8,www.mlk-berlin.de,Q,86.10,"(, Martin-Luther-Krankenhaus - Seit 1931 in Be...",,Martin-Luther-Krankenhaus - Seit 1931 in Berli...,[]
9,www.goldhofer.de,C,29.10,"(, , Goldhofer Unternehmensgruppe, Goldhofer C...",,,"[Goldhofer Unternehmensgruppe, Goldhofer Compa..."


In [10]:
def get_urls_and_keywords(urls, df):
    dict = {}
    
    for url in urls:
        row = metadata_df[metadata_df['url'] == url]
        keywords = row.iloc[0]['keywords']
        dict[url] = keywords
    
    return dict

In [11]:
urls = list(metadata_df['url'])
url_keywords_dict = get_urls_and_keywords(urls, metadata_df)

In [12]:
url_keywords_dict

{'www.hoberg-driesch.de': ['Röhrengrosshandel',
  'Starkwandrohre',
  'Normalwandrohre',
  'Gewinderohre',
  'Kesselrohre',
  'Hohlprofile',
  'Präzisions\xadstahl\xadrohre',
  'HPZ-Rohre',
  'Wälzlager\xadstahl\xadrohre',
  'Feinkornbaustahlrohre',
  'Maschinenbaustahlrohre'],
 'www.hgdf.de': [''],
 'www.gasag.de': ['Strom',
  'Erdgas',
  'Wärme',
  'Privatkunden',
  'Stromvertrag',
  'Erdgasvertrag',
  'Gas',
  'Gasvertrag',
  'Berlin',
  'Brandenburg',
  'Gasversorgung',
  'Gastarif',
  'Stromtarif',
  'Tarif',
  'Geschäftskunden',
  'Kunden',
  'Verbrauch',
  'Energie',
  'Energievertrag',
  'Energieversorgung',
  'Versorgungsunternehmen',
  'Strom wechseln',
  'Gas wechseln'],
 'www.ergodirekt.de': [''],
 'www.wahler.de': [''],
 'www.msa-auer.de': ['MSA',
  'MSA safety',
  'MSAsafety',
  'safety products',
  'msa global'],
 'www.nuernberger.de': ['NÜRNBERGER', 'Versicherung'],
 'www.ds-cc.de': ['Customer Care Services',
  'Kundenservices',
  'Kundenkontakt Management',
  'Call Cen

In [13]:
def remove_https(url):
    www_start = re.search('www', url).start()
    return url[www_start:]

In [14]:
def get_label(url, df):
    row = df[df['url'] == url]
    y = row.iloc[0]['y']
    return y

In [15]:
def clean(words_list):
    clean_list = []
    
    for word in words_list:
        new_word = re.sub('\n+', '', word)
        new_word = re.sub('(\t)+', '', new_word)
        new_word = re.sub('\d+', '', new_word)
        clean_list.append(new_word)
    
    return clean_list

In [16]:
def add_labels_to_dict(dic, local_df):
    new_dic = {}
    count = 0
    for url in dic.keys():
        try:
            y = get_label(url, local_df)
            count += 1
            if '' in dic[url]:
                continue
            clean_phrases = clean(dic[url])
            new_dic[url] = {'text': dic[url], 'label': y}
        except IndexError:
            count += 1
            continue
    return new_dic

In [17]:
# Add labels to train_dict in format {'url': {'text': [words, words], 'label': 1}}
train_data = add_labels_to_dict(url_keywords_dict, df)

In [18]:
# train_data_

In [ ]:
# Words in sequences are replaced by their representing indices in  vocabulary. 
# Unknown words are accouted for with the last index.
def create_ids(data, vocab, maxSeqLength):
    numSentences = len(data)
    ids = np.zeros((numSentences, maxSeqLength), dtype='int32')
    sentenceCounter = 0
    total = 0
    unk_count = 0
    known_words = []
    unk_words = []
    for sentence in data:
        indexCounter = 0
#         print(sentence)
        for word in sentence:
            try:
                ids[sentenceCounter][indexCounter] = vocab.index(word)
                total += 1
                known_words.append(word)
            except:
                ids[sentenceCounter][indexCounter] = len(vocab)-1 # vector for unknown words
                total += 1
                unk_count += 1
                unk_words.append(word)
            indexCounter += 1
            if indexCounter >= maxSeqLength:
                break
        sentenceCounter += 1
    print('Unknown word count \t{}'.format(unk_count))
    print('Total word count \t{}'.format(total))
    print('Percent unknown \t{0:.2f}%'.format(unk_count / total * 100))
    return ids, unk_words, known_words

In [ ]:
def load_data_and_labels(data_dict):
    keys = data_dict.keys()
    
    x_text = []
    y = []
    
    for key in keys:
        x_text.append(data_dict[key]['text'])  # TODO variable size?
        y.append(data_dict[key]['label']) # TODO get category
        
    ###
    ids, unk, known = create_ids(x_text, vocab, 20) # TODO change maxSentSize
    ###
    
    return [ids, y]

In [ ]:
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [ ]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(
      self, sequence_length, num_classes, vocab_size,
      embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):

        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [ ]:
# Parameters
# ==================================================

# Data loading params
# tf.flags.DEFINE_float("dev_sample_percentage", .1, "Percentage of the training data to use for validation")

# Model Hyperparameters
# tf.flags.DEFINE_integer("max_doc_len", 20, "Maximum length of a document: (default: 20)")
# tf.flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of character embedding (default: 128)")
# tf.flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
# tf.flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
# tf.flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
# tf.flags.DEFINE_float("l2_reg_lambda", 0.0, "L2 regularization lambda (default: 0.0)")

# Training parameters
# tf.flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
# tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)")
# tf.flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
# tf.flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
# tf.flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")
# Misc Parameters
# tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
# tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

tf.app.flags.DEFINE_string('f', '', 'kernel')

FLAGS = tf.flags.FLAGS
# FLAGS._parse_flags()
# print("\nParameters:")
# for attr, value in sorted(FLAGS.__flags.items()):
#     print("{}={}".format(attr.upper(), value))
# print("")

def preprocess():
    # Load data
    print("Loading data...")
    x_text, y = load_data_and_labels(train_data) # TODO all data or batch?

    # Build vocabulary
#     max_document_length = max([len(x.split(" ")) for x in x_text])
#     vocab_processor = learn.preprocessing.VocabularyProcessor(20)  # TODO max doc length
#     x = np.array(list(vocab_processor.fit_transform(x_text)))

    # Randomly shuffle data
    np.random.seed(10)
    shuffle_indices = np.random.permutation(np.arange(len(y)))
#     x_shuffled = x[shuffle_indices]
#     x_shuffled = x_text[shuffle_indices]
#     y_shuffled = y[shuffle_indices]

    # Split train/test set
    # TODO: This is very crude, should use cross-validation
#     dev_sample_index = -1 * int(FLAGS.dev_sample_percentage * float(len(y)))
#     x_train, x_dev = x_shuffled[:dev_sample_index], x_shuffled[dev_sample_index:]
#     y_train, y_dev = y_shuffled[:dev_sample_index], y_shuffled[dev_sample_index:]

#     del x, y, x_shuffled, y_shuffled

    print("Vocabulary Size: {:d}".format(len(vocab)))
#     print("Train/Dev split: {:d}/{:d}".format(len(y_train), len(y_dev)))
#     return x_train, y_train, vocab_processor, x_dev, y_dev
    return x_text, np.array([y], dtype=np.int32), None, None

# def train(x_train, y_train, vocab_processor, x_dev, y_dev):
def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto(
          allow_soft_placement=True,
          log_device_placement=False)
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = TextCNN(
                sequence_length=x_train.shape[1],
                num_classes=y_train.shape[1],
#                 vocab_size=len(vocab_processor.vocabulary_),
                vocab_size = len(vocab),
                embedding_size=300,
                filter_sizes=list(map(int, '3,4,5'.split(","))),
                num_filters=128,
                l2_reg_lambda=0.0)

            # Define Training procedure
            global_step = tf.Variable(0, name="global_step", trainable=False)
            optimizer = tf.train.AdamOptimizer(1e-3)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Keep track of gradient values and sparsity (optional)
            grad_summaries = []
            for g, v in grads_and_vars:
                if g is not None:
                    grad_hist_summary = tf.summary.histogram("{}/grad/hist".format(v.name), g)
                    sparsity_summary = tf.summary.scalar("{}/grad/sparsity".format(v.name), tf.nn.zero_fraction(g))
                    grad_summaries.append(grad_hist_summary)
                    grad_summaries.append(sparsity_summary)
            grad_summaries_merged = tf.summary.merge(grad_summaries)

            # Output directory for models and summaries
            timestamp = str(int(time.time()))
            out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
            print("Writing to {}\n".format(out_dir))

            # Summaries for loss and accuracy
            loss_summary = tf.summary.scalar("loss", cnn.loss)
            acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

            # Train Summaries
            train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
            train_summary_dir = os.path.join(out_dir, "summaries", "train")
            train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

            # Dev summaries
            dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
            dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
            dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

            # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
            checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
            checkpoint_prefix = os.path.join(checkpoint_dir, "model")
            if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)
            saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)

            # Write vocabulary
#             vocab_processor.save(os.path.join(out_dir, "vocab"))

            # Initialize all variables
            sess.run(tf.global_variables_initializer())

            def train_step(x_batch, y_batch):
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 0.5
                }
                _, step, summaries, loss, accuracy = sess.run(
                    [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                train_summary_writer.add_summary(summaries, step)

            def dev_step(x_batch, y_batch, writer=None):
                """
                Evaluates model on a dev set
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y: y_batch,
                  cnn.dropout_keep_prob: 1.0
                }
                step, summaries, loss, accuracy = sess.run(
                    [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
                if writer:
                    writer.add_summary(summaries, step)

            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train)), 64, 200)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch = zip(*batch)
                train_step(x_batch, y_batch)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 100 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev, writer=dev_summary_writer)
                    print("")
                if current_step % 100 == 0:
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                    print("Saved model checkpoint to {}\n".format(path))

def main(argv=None):
#     x_train, y_train, vocab_processor, x_dev, y_dev = preprocess()
#     train(x_train, y_train, vocab_processor, x_dev, y_dev)
    x_train, y, x_dev, y_dev = preprocess()
    train(x_train, y, x_dev, y_dev)

In [ ]:
# Training
main()

Loading data...
